### SmartScrape Project

In [42]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
load_dotenv()



# loading env varables
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")



In [43]:
"GOOGLE_API_KEY" in os.environ

True

In [44]:
page_url = "https://python.langchain.com/docs/how_to/chatbots_memory/"
loader = WebBaseLoader(web_paths=[page_url])

In [45]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://python.langchain.com/docs/how_to/chatbots_memory/', 'title': 'How to add memory to chatbots | 🦜️🔗 LangChain', 'description': 'A key feature of chatbots is their ability to use the content of previous conversational turns as context. This state management can take several forms, including:', 'language': 'en'}, page_content='\n\n\n\n\nHow to add memory to chatbots | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Par

In [46]:
# splitting of text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100,
    
)

texts = text_splitter.split_documents(docs)

In [47]:
texts

[Document(metadata={'source': 'https://python.langchain.com/docs/how_to/chatbots_memory/', 'title': 'How to add memory to chatbots | 🦜️🔗 LangChain', 'description': 'A key feature of chatbots is their ability to use the content of previous conversational turns as context. This state management can take several forms, including:', 'language': 'en'}, page_content='How to add memory to chatbots | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/how_to/chatbots_memory/', 'title': 'How to add memory to chatbots | 🦜️🔗 LangChain', 'description': 'A key feature of chatbots is their ability to use the content of previous conversational turns as context. This state management can take several forms, including:', 'language': 'en'}, page_content='Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.

In [48]:
# google chatmodel 
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
)

# google Embedding Model 
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [49]:
# vector storage 

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
    )

vector_store.add_documents(documents=texts)

['9ad631fc-9129-4140-9ba1-55a0bbb8c160',
 '47bfa3f3-f4c8-4411-93ac-ada690c63ca7',
 '676198c6-1c94-4ec5-bc0a-00e8790e5b15',
 'f77a6543-2514-4970-abfc-c701d9c90b5b',
 'f97c5653-3faa-4d1d-8fef-25c933bcf321',
 '257677cd-b236-456a-bb13-4271478c7f93',
 'a10cd597-d7a8-4b42-ba06-1668886d1ad0',
 '9cfe0c07-2c0c-4880-9583-dedac37c2423',
 '7fe5a600-03db-44e4-a346-5e99f9b8ff68',
 '94998ddf-63bc-4f7c-a641-7af6e15f1971',
 '8a22d507-ffec-484d-acc4-f13128839e71',
 '551eb66d-4201-4b7c-b1bb-bc888371b8ce',
 '833981dd-88e6-4db5-afd8-a78d1daed105',
 '378a3283-9d66-48bc-983e-45ac424f709e',
 '991e9fb2-2691-4306-b4d7-577db2e178fc',
 'c7f57e06-38b5-48d9-ac99-1f9a644ee9fb',
 '9cdbf20b-8b29-41d6-972e-1926c5c5d348',
 'ca293178-2695-47ba-9a05-17afe8b7efab',
 '44f8a989-0595-4732-bf88-8e22f6cc68db',
 '67ff2dea-a059-4c7a-90b8-69e13cd7b798',
 '8de409d5-4462-439b-b50d-e0294b932f5b',
 '532a38d4-726c-4bab-bc56-0ba7056ce152',
 'f08ddfca-e8f3-46e8-a237-2597bdd189bb',
 'c7910ceb-92fe-49cb-a176-9ec43b7781e9',
 '874bc4de-894a-

In [50]:
query ="Automatic history management"
print(vector_store.similarity_search(query)[0].page_content)

Automatic history management​
The previous examples pass messages to the chain (and model) explicitly. This is a completely acceptable approach, but it does require external management of new messages. LangChain also provides a way to build applications that have memory using LangGraph's persistence. You can enable persistence in LangGraph applications by providing a checkpointer when compiling the graph.


In [51]:
retriver = vector_store.as_retriever()
retriver.invoke(query)

[Document(id='ca8d2e2b-eba0-4152-87c2-8eeac9ac3371', metadata={'source': 'https://python.langchain.com/docs/how_to/chatbots_memory/', 'title': 'How to add memory to chatbots | 🦜️🔗 LangChain', 'description': 'A key feature of chatbots is their ability to use the content of previous conversational turns as context. This state management can take several forms, including:', 'language': 'en'}, page_content="Automatic history management\u200b\nThe previous examples pass messages to the chain (and model) explicitly. This is a completely acceptable approach, but it does require external management of new messages. LangChain also provides a way to build applications that have memory using LangGraph's persistence. You can enable persistence in LangGraph applications by providing a checkpointer when compiling the graph."),
 Document(id='e4122133-a0c7-4a1f-9aa1-285b0123bc2b', metadata={'source': 'https://python.langchain.com/docs/how_to/chatbots_memory/', 'title': 'How to add memory to chatbots |

In [56]:
# This chain takes a list of documents and formats them all into a prompt, then passes that prompt to an LLM. 


from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.prompts import ChatPromptTemplate
# 2. Create a prompt template
prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant. Given the following context, answer the question.

Context:
{context}

Question: {question}
""")

# prompt = ChatPromptTemplate([
#     ("system","your assitance")
# ])

chain = create_stuff_documents_chain(llm,prompt)
chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nYou are a helpful assistant. Given the following context, answer the question.\n\nContext:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7a3f50277ef0>, default_metadata=(), model_kwargs={})
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [54]:
# 5. Run the chain
response = chain.invoke({
    "question": "what is Automatic history management",
    "context": retriver.invoke(query)
})


In [55]:
print(response)

Automatic history management in LangChain refers to the ability to build applications with memory using LangGraph's persistence. This allows the application to automatically keep track of and manage the history of messages, accumulating them until a specified number is reached. When the limit is reached, a summary of the history is generated and used to continue the conversation. This is enabled by providing a checkpointer when compiling the graph.
